In [1]:
import numpy as np
import pandas as pd
import os
import math
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt

In [28]:
paths = []

for i in range(1, 16):
    root, file = os.path.split("../Dataset/SA01/F01_SE06_R01.csv")
    if i < 10:
        file = file[:2] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        paths.append(tmp)
        if i > 4:
            for j in range(2, 6):
                file_new = file[:-5] + str(j) + file[-4:]
                tmp = os.path.join(root, file_new)
                paths.append(tmp)
    if i >= 10:
        file = file[:1] + str(i) + file[3:]
        for j in range(1, 6):
            file = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file)
            paths.append(tmp)

# paths[4:14] = ['../Dataset/SA01/D05_SA01_R01_1.csv',
#                 '../Dataset/SA01/D05_SA01_R01_2.csv',
#                 '../Dataset/SA01/D05_SA01_R02_1.csv',
#                 '../Dataset/SA01/D05_SA01_R02_2.csv',
#                 '../Dataset/SA01/D05_SA01_R03_1.csv',
#                 '../Dataset/SA01/D05_SA01_R03_2.csv',
#                 '../Dataset/SA01/D05_SA01_R04_1.csv',
#                 '../Dataset/SA01/D05_SA01_R04_2.csv',
#                 '../Dataset/SA01/D05_SA01_R05_1.csv',
#                 '../Dataset/SA01/D05_SA01_R05_2.csv',
#                 '../Dataset/SA01/D06_SA01_R01_1.csv',
#                 '../Dataset/SA01/D06_SA01_R01_2.csv',
#                 '../Dataset/SA01/D06_SA01_R02_1.csv',
#                 '../Dataset/SA01/D06_SA01_R02_2.csv',
#                 '../Dataset/SA01/D06_SA01_R03_1.csv',
#                 '../Dataset/SA01/D06_SA01_R03_2.csv',
#                 '../Dataset/SA01/D06_SA01_R04_1.csv',
#                 '../Dataset/SA01/D06_SA01_R04_2.csv',
#                 '../Dataset/SA01/D06_SA01_R05_1.csv',
#                 '../Dataset/SA01/D06_SA01_R05_2.csv']

# paths[-15:-10] = ['../Dataset/SA01/D17_SA01_R01_1.csv',
#                  '../Dataset/SA01/D17_SA01_R01_2.csv',
#                  '../Dataset/SA01/D17_SA01_R02_1.csv',
#                  '../Dataset/SA01/D17_SA01_R02_2.csv',
#                  '../Dataset/SA01/D17_SA01_R03_1.csv',
#                  '../Dataset/SA01/D17_SA01_R03_2.csv',
#                  '../Dataset/SA01/D17_SA01_R04_1.csv',
#                  '../Dataset/SA01/D17_SA01_R04_2.csv',
#                  '../Dataset/SA01/D17_SA01_R05_1.csv',
#                 '../Dataset/SA01/D17_SA01_R05_2.csv']

SE06_F_paths = []

for path in paths:
    path = path.replace('SA01', 'SE06')
    SE06_F_paths.append(path)

SE06_F_paths

['../Dataset/SE06\\F01_SE06_R01.csv',
 '../Dataset/SE06\\F02_SE06_R01.csv',
 '../Dataset/SE06\\F03_SE06_R01.csv',
 '../Dataset/SE06\\F04_SE06_R01.csv',
 '../Dataset/SE06\\F05_SE06_R01.csv',
 '../Dataset/SE06\\F05_SE06_R02.csv',
 '../Dataset/SE06\\F05_SE06_R03.csv',
 '../Dataset/SE06\\F05_SE06_R04.csv',
 '../Dataset/SE06\\F05_SE06_R05.csv',
 '../Dataset/SE06\\F06_SE06_R01.csv',
 '../Dataset/SE06\\F06_SE06_R02.csv',
 '../Dataset/SE06\\F06_SE06_R03.csv',
 '../Dataset/SE06\\F06_SE06_R04.csv',
 '../Dataset/SE06\\F06_SE06_R05.csv',
 '../Dataset/SE06\\F07_SE06_R01.csv',
 '../Dataset/SE06\\F07_SE06_R02.csv',
 '../Dataset/SE06\\F07_SE06_R03.csv',
 '../Dataset/SE06\\F07_SE06_R04.csv',
 '../Dataset/SE06\\F07_SE06_R05.csv',
 '../Dataset/SE06\\F08_SE06_R01.csv',
 '../Dataset/SE06\\F08_SE06_R02.csv',
 '../Dataset/SE06\\F08_SE06_R03.csv',
 '../Dataset/SE06\\F08_SE06_R04.csv',
 '../Dataset/SE06\\F08_SE06_R05.csv',
 '../Dataset/SE06\\F09_SE06_R01.csv',
 '../Dataset/SE06\\F09_SE06_R02.csv',
 '../Dataset

In [27]:
def butter_lowpass(cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order=order)
    x = np.trunc(np.array(data, dtype=np.float64)).astype(np.int64)
    y = lfilter(b, a, x, axis=-1, zi=None)
    return y

# Setting standard filter requirements.
order = 4
fs = 200
cutoff = 5

b, a = butter_lowpass(cutoff, fs, order)

# Creating the data for filteration
T = 200         # value taken in seconds
n = int(T * fs) # indicates total samples
t = np.linspace(0, T, n, endpoint=False)

In [52]:
list = []
LPF_list = []
final_SA23_F = pd.DataFrame(list)

for path in SA23_F_paths:
    try:
        data = pd.read_csv(path, header=None)
        data = data.iloc[:, :3]
        rows, cols = data.shape

        sliced_df = data[0 : 160]
    
        # LPF 구하기

        col_1_lpf = butter_lowpass_filter(sliced_df[0].to_numpy(), cutoff, fs, order).astype('float64')
        col_2_lpf = butter_lowpass_filter(sliced_df[1].to_numpy(), cutoff, fs, order).astype('float64')
        col_3_lpf = butter_lowpass_filter(sliced_df[2].to_numpy(), cutoff, fs, order).astype('float64')
    
        # 가속도 구하기
        # 표준편차 구하기

        col_1 = col_1_lpf*col_1_lpf
        col_2 = col_2_lpf*col_2_lpf
        col_3 = col_3_lpf*col_3_lpf
    
        C_nine = np.sqrt(col_1 + col_2 + col_3)
        list_C_nine = pd.DataFrame(C_nine).transpose().round(3)
        final_SA23_F = final_SA23_F.append(list_C_nine)
        
    except:
        pass

pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

row, col = final_SA23_F.shape
target_col = row * [1]
final_SA23_F.loc[:,'target'] = target_col

final_SA23_F

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,target
0,0.008,0.069,0.296,0.871,2.019,3.966,6.924,11.069,16.530,23.384,31.655,41.313,52.279,64.433,77.621,91.658,106.342,121.458,136.784,152.098,167.187,181.852,195.912,209.208,221.605,232.995,243.297,252.455,260.442,267.249,272.889,277.391,280.805,283.193,284.632,285.212,285.032,284.195,282.807,280.975,278.800,276.379,273.804,271.159,268.520,265.953,263.516,261.258,259.213,257.408,...,248.388,248.829,249.346,249.940,250.613,251.373,252.225,253.175,254.229,255.391,256.658,258.027,259.484,261.012,262.587,264.182,265.764,267.301,268.759,270.104,271.303,272.324,273.139,273.726,274.067,274.153,273.978,273.545,272.858,271.932,270.784,269.439,267.926,266.276,264.522,262.695,260.825,258.938,257.058,255.204,253.393,251.639,249.958,248.364,246.874,245.506,244.275,243.201,242.302,1
0,0.008,0.068,0.292,0.858,1.986,3.898,6.801,10.871,16.237,22.977,31.119,40.639,51.463,63.476,76.528,90.438,105.005,120.015,135.247,150.479,165.499,180.106,194.117,207.371,219.732,231.088,241.357,250.479,258.423,265.182,270.769,275.220,278.589,280.945,282.370,282.956,282.804,282.020,280.711,278.986,276.949,274.700,272.334,269.934,267.578,265.327,263.236,261.343,259.677,258.254,...,247.924,248.857,249.939,251.176,252.569,254.119,255.820,257.661,259.624,261.686,263.815,265.978,268.136,270.248,272.277,274.185,275.939,277.509,278.871,280.005,280.894,281.528,281.897,281.997,281.828,281.391,280.693,279.742,278.549,277.130,275.501,273.685,271.707,269.595,267.380,265.095,262.772,260.443,258.139,255.891,253.726,251.670,249.748,247.981,246.387,244.980,243.770,242.766,241.972,1
0,0.008,0.069,0.296,0.873,2.021,3.970,6.930,11.076,16.539,23.398,31.677,41.347,52.330,64.504,77.709,91.757,106.441,121.540,136.828,152.083,167.092,181.658,195.602,208.767,221.023,232.265,242.414,251.420,259.257,265.925,271.447,275.865,279.237,281.634,283.140,283.845,283.842,283.227,282.093,280.532,278.630,276.468,274.122,271.663,269.155,266.655,264.216,261.882,259.694,257.685,...,249.381,250.069,250.865,251.758,252.744,253.815,254.969,256.204,257.521,258.920,260.403,261.968,263.609,265.318,267.079,268.867,270.655,272.407,274.083,275.641,277.036,278.220,279.152,279.791,280.107,280.074,279.679,278.919,277.799,276.337,274.562,272.509,270.220,267.743,265.130,262.436,259.717,257.030,254.429,251.962,249.672,247.592,245.748,244.156,242.824,241.751,240.929,240.344,239.977,1
0,0.008,0.068,0.294,0.865,2.003,3.931,6.856,10.950,16.339,23.097,31.250,40.771,51.586,63.582,76.607,90.483,105.015,119.996,135.214,150.457,165.520,180.208,194.344,207.768,220.343,231.954,242.511,251.952,260.239,267.361,273.331,278.181,281.963,284.741,286.591,287.596,287.847,287.438,286.466,285.024,283.204,281.093,278.771,276.313,273.782,271.232,268.710,266.252,263.889,261.645,...,255.227,255.326,255.412,255.479,255.523,255.541,255.529,255.485,255.410,255.305,255.170,255.010,254.828,254.629,254.422,254.218,254.029,253.867,253.742,253.662,253.633,253.657,253.732,253.855,254.018,254.211,254.424,254.644,254.862,255.070,255.263,255.437,255.593,255.734,255.865,255.990,256.116,256.248,256.392,256.550,256.724,256.911,257.111,257.319,257.529,257.734,257.927,258.100,258.248,1
0,0.008,0.069,0.297,0.876,2.027,3.980,6.945,11.098,16.569,23.438,31.727,41.410,52.410,64.607,77.845,91.943,106.695,121.884,137.287,152.679,167.843,182.571,196.676,209.989,222.365,233.685,243.861,252.833,260.572,267.075,272.362,276.478,279.485,281.461,282.497,282.693,282.154,280.985,279.292,277.179,274.747,272.091,269.305,266.474,263.677,260.985,258.462,256.160,254.122,252.376,...,255.559,254.946,254.290,253.608,252.916,252.225,251.547,250.885,250.240,249.604,248.968,248.315,247.623,246.869,246.026,245.070,243.981,242.747,241.366,239.849,2

In [53]:
A01 = final_SA01_F

In [54]:
A02 = final_SA02_F

In [55]:
A03 = final_SA03_F

In [56]:
A04 = final_SA04_F

In [57]:
A05 = final_SA05_F

In [58]:
A06 = final_SA06_F

In [59]:
A07 = final_SA07_F

In [60]:
A08 = final_SA08_F

In [61]:
A09 = final_SA09_F

In [62]:
A10 = final_SA10_F

In [63]:
A11 = final_SA11_F

In [64]:
A12 = final_SA12_F

In [65]:
A13 = final_SA13_F

In [66]:
A14 = final_SA14_F

In [67]:
A15 = final_SA15_F

In [68]:
A16 = final_SA16_F

In [69]:
A17 = final_SA17_F

In [70]:
A18 = final_SA18_F

In [71]:
A19 = final_SA19_F

In [72]:
A20 = final_SA20_F

In [73]:
A21 = final_SA21_F

In [74]:
A22 = final_SA22_F

In [75]:
A23 = final_SA23_F

In [76]:
E06 = final_SE06_F

In [77]:
FALL_all = pd.concat([A01, A02, A03, A04, A05, A06, A07, A08, A09, A10, A11, A12, A13, A14, A15, A16, A17, A18, A19, A20, A21, A22, A23, E06])

In [78]:
FALL_all.to_csv('[C1]FALL_LPF.csv',index=False)

In [79]:
FALL_all.shape

(1416, 161)

In [80]:
df_1 = pd.read_csv('[C1]ADL_LPF.csv')
df_2 = pd.read_csv('[C1]FALL_LPF.csv')

In [81]:
df = pd.concat([df_1,df_2])

In [84]:
df.to_csv('res_CLPF.csv', index=False)

In [85]:
df.shape

(4720, 161)